In [23]:
import pandas as pd
import sklearn
import numpy as np
import joblib

In [2]:
test_data22 = pd.read_csv('data/test.csv')
singl = test_data22[5:6]
singl['Cabin'].fillna(value ='C85',inplace = True,axis = 0)

singl

C:\Users\Khan\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.225,C85,S


In [13]:
# import the BaseEstimator
from sklearn.base import BaseEstimator

class DropColumns(BaseEstimator):
    def __init__(self, cols):
        self.cols = cols
        print(self.cols)
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        
        X.drop(self.cols,axis = 1, inplace = True)
        return X
    
#===============================Cabin Columns Feature Engineerong
class CabinTransform(BaseEstimator):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, data):
        cabinValues = [1,2,3,4,5,6,7,8]
        deck = data['Cabin'].dropna(axis = 0).str[0:1].unique()
        cabinKey = np.sort(deck, kind = 'quick')
        self.deckPair = {cabinKey[i]: cabinValues[i] for i in range(len(cabinKey))} 
        data['Cabin'].fillna('0',inplace = True)
        data['Deck'] = data['Cabin'].apply(lambda x : self.setDeck(x))
        data['Deck'] = data['Deck'].astype('int32')
        return data
    
    def setDeck(self,x):
        char1 = x[0:1]
        if char1 == '0':
            return 0
        else:
            return self.deckPair[char1]


    

In [14]:
class AgeandFareTransform(BaseEstimator):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, data):
        if len(data) == 1:
            data['Fare'].fillna(value = data['Fare'].mean(),axis = 0,inplace = True) ## fillNa with with
            data['Age'].fillna(value = data['Age'].mean(),axis = 0,inplace = True) ## fillNa with with
            data['Age'] = data['Age'].astype('int32')
            return data
        else:    
            nullIndex = data['Age'].index[data['Age'].apply(np.isnan)]
            rand_age = self.getStatistic(data)
            sr1 = pd.Series(rand_age, index = nullIndex) # It is necessary to set indexof NaN to series,so that it can set values according to index
            data['Age'].fillna(value = sr1,axis = 0,inplace = True) ## fillNa with with same length and on same index
            data['Age'] = data['Age'].astype('int32')
            data['Fare'].fillna(value = data['Fare'].mean(),axis = 0,inplace = True) ## fillNa with with same length and on same index
            return data
    
    def getStatistic(self,data):
        mean = data['Age'].mean()
        std = data['Age'].std()
        is_null = data["Age"].isnull().sum()
#         print("Mean:{mean},Std:{std},isNull:{is_null}".format(mean = mean,std = std,is_null = is_null))
        rand_age = np.random.randint(mean - std, mean + std, size = is_null)
        return rand_age
    

class filEmbark(BaseEstimator):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    def transform(self, data):
        data['Embarked'].fillna(value = 'S',axis = 0,inplace = True)
        ports = {"S": 0, "C": 1, "Q": 2}
        data['Embarked'] = data['Embarked'].map(ports)
        return data
    
class Gender(BaseEstimator):
    def __init__(self):
        self.genders = {"male": 0, "female": 1}
        
    def fit(self, X, y=None):
        return self
    def transform(self, data):
        data['Sex'] = data['Sex'].map(self.genders)
        return data
    
class SibspParch(BaseEstimator):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, data):
        data['relatives'] = data['SibSp'] + data['Parch']
        data.loc[data['relatives'] > 0,'not_alone'] = 1
        data.loc[data['relatives'] == 0,'not_alone'] = 0
        return data

In [30]:
# # Import JOblib
cleanDataPipeline = joblib.load("clean_data.joblib")
cleanDataPipeline

Pipeline(memory=None,
     steps=[('setCabin', CabinTransform()), ('setAge', AgeandFareTransform()), ('filEmbark', filEmbark()), ('Gender', Gender()), ('Relatives', SibspParch()), ('dropColumns', DropColumns(cols=['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch', 'not_alone']))])

In [24]:
cleanDataPipeline.fit_transform(singl) 
# # machinePipeline
singl

C:\Users\Khan\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Khan\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Khan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

,Pclass,Sex,Age,Fare,Embarked,Deck,relatives
5,3,0,14,9.225,0,1,0


In [26]:
SVM_model= joblib.load("SVM_Model.joblib")
pred = SVM_model.predict(singl)
pred

C:\Users\Khan\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


array([0], dtype=int64)

In [31]:
pred[0]

0